In [3]:
# !pip install langchain chromadb openai tiktoken pypdf langchain_openai langchain-community

In [ ]:
!pip install google-generativeai langchain chromadb --quiet

import google.generativeai as genai
from langchain.vectorstores import Chroma
from langchain.embeddings.base import Embeddings

genai.configure(api_key="YOUR_API_KEY")

class GeminiEmbeddingFunction(Embeddings):
    def embed_documents(self, texts):
        return [
            genai.embed_content(
                model="models/embedding-001",
                content=text,
                task_type="retrieval_document"
            )["embedding"] for text in texts
        ]
    def embed_query(self, text):
        return genai.embed_content(
            model="models/embedding-001",
            content=text,
            task_type="retrieval_query"
        )["embedding"]


In [5]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [6]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [7]:
vector_store = Chroma(
    embedding_function=GeminiEmbeddingFunction(),
    persist_directory='my_chroma_db',
    collection_name='sample'
)

/tmp/ipython-input-7-337313879.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [8]:
# add documents
vector_store.add_documents(docs)

['58347991-ad64-46d5-95a0-4b6a4aca228f',
 '341e9d70-4012-4cf4-9302-f13c5c22e0de',
 'd0257ea8-4603-41c7-8c1e-3ceea72e0d20',
 'de9215d1-60ef-4bf4-a03d-d8aaab6e72c2',
 'dbc7a3a6-9e23-436f-9154-4eef77fa90c3']

In [9]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['58347991-ad64-46d5-95a0-4b6a4aca228f',
  '341e9d70-4012-4cf4-9302-f13c5c22e0de',
  'd0257ea8-4603-41c7-8c1e-3ceea72e0d20',
  'de9215d1-60ef-4bf4-a03d-d8aaab6e72c2',
  'dbc7a3a6-9e23-436f-9154-4eef77fa90c3'],
 'embeddings': array([[ 1.35614481e-02, -7.62679800e-03, -2.51727290e-02, ...,
          1.35761239e-02,  3.51136659e-05, -7.35754846e-03],
        [ 3.18830460e-02, -3.36795598e-02, -4.39120680e-02, ...,
          1.08157219e-02, -1.05898809e-02, -1.52005616e-03],
        [ 1.58164389e-02,  3.38992523e-03, -2.23509353e-02, ...,
          1.95286470e-03, -7.18898326e-02, -8.88233818e-03],
        [ 1.01624681e-02, -5.16264960e-02, -2.89599448e-02, ...,
          3.29840817e-02, -2.09922232e-02,  1.78275332e-02],
        [-8.94617348e-04,  1.97164640e-02, -9.64438636e-03, ...,
          2.34126463e-03, -1.33365402e-02, -1.25717660e-02]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style 

In [12]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.')]

In [13]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.770696222782135),
 (Document(metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.'),
  0.8030722737312317)]

In [15]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query=" ",
    filter={"team": "Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.871087908744812),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.8778100609779358)]

In [16]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)


In [17]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['58347991-ad64-46d5-95a0-4b6a4aca228f',
  '341e9d70-4012-4cf4-9302-f13c5c22e0de',
  'd0257ea8-4603-41c7-8c1e-3ceea72e0d20',
  'de9215d1-60ef-4bf4-a03d-d8aaab6e72c2',
  'dbc7a3a6-9e23-436f-9154-4eef77fa90c3'],
 'embeddings': array([[ 1.35614481e-02, -7.62679800e-03, -2.51727290e-02, ...,
          1.35761239e-02,  3.51136659e-05, -7.35754846e-03],
        [ 3.18830460e-02, -3.36795598e-02, -4.39120680e-02, ...,
          1.08157219e-02, -1.05898809e-02, -1.52005616e-03],
        [ 1.58164389e-02,  3.38992523e-03, -2.23509353e-02, ...,
          1.95286470e-03, -7.18898326e-02, -8.88233818e-03],
        [ 1.01624681e-02, -5.16264960e-02, -2.89599448e-02, ...,
          3.29840817e-02, -2.09922232e-02,  1.78275332e-02],
        [-8.94617348e-04,  1.97164640e-02, -9.64438636e-03, ...,
          2.34126463e-03, -1.33365402e-02, -1.25717660e-02]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style 

In [18]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [19]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['58347991-ad64-46d5-95a0-4b6a4aca228f',
  '341e9d70-4012-4cf4-9302-f13c5c22e0de',
  'd0257ea8-4603-41c7-8c1e-3ceea72e0d20',
  'de9215d1-60ef-4bf4-a03d-d8aaab6e72c2',
  'dbc7a3a6-9e23-436f-9154-4eef77fa90c3'],
 'embeddings': array([[ 1.35614481e-02, -7.62679800e-03, -2.51727290e-02, ...,
          1.35761239e-02,  3.51136659e-05, -7.35754846e-03],
        [ 3.18830460e-02, -3.36795598e-02, -4.39120680e-02, ...,
          1.08157219e-02, -1.05898809e-02, -1.52005616e-03],
        [ 1.58164389e-02,  3.38992523e-03, -2.23509353e-02, ...,
          1.95286470e-03, -7.18898326e-02, -8.88233818e-03],
        [ 1.01624681e-02, -5.16264960e-02, -2.89599448e-02, ...,
          3.29840817e-02, -2.09922232e-02,  1.78275332e-02],
        [-8.94617348e-04,  1.97164640e-02, -9.64438636e-03, ...,
          2.34126463e-03, -1.33365402e-02, -1.25717660e-02]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style 